# stpe1 - 利用LLM来对图片进行分析

In [ ]:
# 读取一个文件夹,


import os
import base64
import requests
import json
import pandas as pd
import ujson as json
from openai import OpenAI
import os



# 设置API密钥和基础URL
api_key = "35f54cc4-be7a-4414-808e-f5f9f0194d4f"
api_base = "http://gpt-proxy.jd.com/v1/chat/completions"
client = OpenAI(api_key=api_key, base_url=api_base)



# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image folder
image_folder = "D://code//data//background_color//服饰鞋靴箱包//女士春夏上装//grounding_output//test1"

# Initialize a list to store the results
results = []

# Loop through each image in the folder
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_folder, filename)
        base64_image = encode_image(image_path)

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }

        payload = {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": '''
                                    你是一个优秀的设计师和数据分析师. 请描述画面的整体类型, 需要包含如下维度:
                                    1.价促类型(只要出现价格信息/促销信息/满减/买赠等营销活动信息),功能类型(以商品功能/卖点/细节等信息为主),白底图(背景为白色的图片),场景图(背景为非白底图);
                                    2.请描述画面中背景的视觉风格/色彩/背景中包含的视觉元素/是否模糊/背景类型风格
                                    3.请描述画面中是否存在logo
                                    4.请描述是否存在除商品主体外的小元素, 小元素的定义是指商品主体和logo以外的非文字类的元素(特别注意不要统计文字类),被单独分割出来,通常周边会填充颜色以和背景区别, 如: 图案/图标/形状/等, 并给出矩形框坐标;
                                    5.请描述是否存在除商品主体外的细节图,细节图的定义是指商品主体以外的元素, 被单独分割出来, 通常周边会填充颜色以和背景区别, 如:放大的局部/材质描述/功能描述等,并给出矩形框坐标

                                    以上结果需要按照统一的格式输出: 
                                    画面类型(价促类or功能类or白底图or场景图);
                                    商品主体(具体的商品主体描述, 坐标(x1,y1), (x2,y2)); 
                                    视觉风格(具体的视觉风格描述); 色彩(具体色彩描述); 
                                    元素(具体的元素描述);
                                    是否模糊(模糊或清晰);
                                    小元素(是or否,描述元素内容,给出矩形框坐标);
                                    细节图(是or否,描述细节内容,给出矩形框坐标) ; 
                                    画面内容(具体的文字描述, 如果是前面已经出现过的内容就不要描述);
                                    '''

                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 700
        }

        response = requests.post("http://gpt-proxy.jd.com/v1/chat/completions", headers=headers, json=payload)
        # print(response.json())

        result = response.json()
        generated_text = result['choices'][0]['message']['content']
        # print(generated_text)

    # Create a dictionary with the image filename and generated text
        image_result = {"filename": filename, "generated_text": generated_text}
        results.append(image_result)



# # Save the results as a JSON file
# with open('D://code//data//background_color//服饰鞋靴箱包//女士春夏上装//grounding_output//test1//image_results.json', 'w') as f:
#     json.dump(results, f, indent=4)

# Save the results as an Excel file
df = pd.DataFrame(results)
df.to_excel('D://code//data//background_color//服饰鞋靴箱包//女士春夏上装//grounding_output//test1//image_results.xlsx', index=False)





# step2 - 将图片进行分割

In [ ]:
# import os
# import numpy as np
# import torch
# import gc
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# from torchvision.ops import box_convert
# from groundingdino.util.inference import load_model, load_image, predict, annotate
# from segment_anything import sam_model_registry, SamPredictor
# import openpyxl

# # 忽略特定警告
# import warnings
# warnings.filterwarnings("ignore", category=FutureWarning, message=".*device.*")
# warnings.filterwarnings("ignore", category=UserWarning, message=".*requires_grad.*")

# # 使用 GPU 设备（如果可用）
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Using device: {device}")

# # 加载 GroundingDINO 模型
# cfg_path = r'D://code//CV//GroundingDINO//groundingdino//config//GroundingDINO_SwinT_OGC.py'
# weight_path = r'd://code//CV//GroundingDINO//weight//groundingdino_swint_ogc.pth'
# gdino_model = load_model(cfg_path, weight_path)
# gdino_model.to(device)
# BOX_THRESHOLD = 0.35
# TEXT_THRESHOLD = 0.35

# # 加载 Segment Anything 模型
# sam_checkpoint = r"D://aigc//ComfyUI_windows_portable//ComfyUI//models//sams//sam_vit_h_4b8939.pth"
# model_type = "vit_h"
# sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
# sam.to(device)


# def process_images_in_folder_structure(input_folder, text_prompt):
#     total_files = 0
#     processed_files = 0

#     # 获取二级文件夹列表
#     second_level_folders = [f.path for f in os.scandir(input_folder) if f.is_dir()]

#     for second_folder in second_level_folders:
#         print(f"Processing second level folder: {second_folder}")
        
#         # 获取三级文件夹列表
#         third_level_folders = [f.path for f in os.scandir(second_folder) if f.is_dir()]

#         for third_folder in third_level_folders:
#             print(f"Processing third level folder: {third_folder}")
            
#             output_folder = os.path.join(third_folder, 'grounding_output')
#             if not os.path.exists(output_folder):
#                 os.makedirs(output_folder)

#             image_files = [f for f in os.listdir(third_folder) if f.lower().endswith((".jpg", ".png"))]
#             total_files += len(image_files)

#             for filename in tqdm(image_files, desc=f"Processing {third_folder}"):
#                 ip = os.path.join(third_folder, filename)
#                 try:
#                     image, boxes = get_box_from_gdino_with_text_prompt(ip, text_prompt)
#                     if image is not None and boxes is not None:
#                         seg_with_sam_with_box_prompt(image, boxes, text_prompt, output_folder, ip)
#                         processed_files += 1
#                 except Exception as e:
#                     print(f"处理文件 {filename} 时出错: {e}")
#                 finally:
#                     plt.close('all')
#                     gc.collect()

#     print(f"总文件数: {total_files}")
#     print(f"成功处理的文件数: {processed_files}")


# def get_box_from_gdino_with_text_prompt(ip, text_prompt):
#     try:
#         image_source, image = load_image(ip)
#         if image_source is None or image is None:
#             print(f"无法加载图像: {ip}")
#             return None, None

#         h, w = image_source.shape[:2]
#         boxes, logits, phrases = predict(
#             model=gdino_model,
#             image=image,
#             caption=text_prompt,
#             box_threshold=BOX_THRESHOLD,
#             text_threshold=TEXT_THRESHOLD,
#             device=device
#         )

#         # 确保 boxes 在正确的设备上
#         boxes = boxes.to(device)

#         # 创建缩放张量并移动到正确的设备
#         scale_tensor = torch.tensor([w, h, w, h], dtype=boxes.dtype, device=device)

#         # 进行缩放操作
#         boxes = boxes * scale_tensor

#         # 转换坐标格式
#         xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").cpu().numpy()

#         del image, logits, phrases, boxes, scale_tensor
#         gc.collect()
#         return image_source, xyxy
#     except Exception as e:
#         print(f"加载图像 {ip} 时出错: {e}")
#         return None, None


# def seg_with_sam_with_box_prompt(image, boxes, text_prompt, output_folder, ip):
#     predictor = SamPredictor(sam)
#     predictor.set_image(image)
#     merged_mask = None

#     for idx, box in enumerate(boxes):
#         input_box = box.astype(int)
#         masks, _, _ = predictor.predict(
#             point_coords=None,
#             point_labels=None,
#             box=input_box[None, :],
#             multimask_output=False,
#         )
#         if masks is not None and len(masks) > 0:
#             if merged_mask is None:
#                 merged_mask = masks[0]
#             else:
#                 merged_mask += masks[0]

#     if merged_mask is not None:
#         merged_mask = np.where(merged_mask > 0, 1, 0)
#         if np.any(merged_mask):
#             base_name = os.path.basename(ip).split('.')[0]
#             output_filename = f"{base_name}.jpg"
#             output_path = os.path.join(output_folder, output_filename)
#             save_masked_image(image, merged_mask, output_path, boxes)

#             # 记录四个顶点的坐标
#             vertex_coords = [box for box in boxes]

#             # 将图片名和坐标保存到 Excel 文件中
#             base_name = os.path.basename(ip).split('.')[0]
#             excel_file = os.path.join(output_folder, 'grounding_results.xlsx')
#             if not os.path.exists(excel_file):
#                 workbook = openpyxl.Workbook()
#                 worksheet = workbook.active
#                 worksheet.append(['Image Name', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])
#             else:
#                 workbook = openpyxl.load_workbook(excel_file)
#                 worksheet = workbook.active

#             seen_vertices = set()
#             for i, box in enumerate(boxes):
#                 # 检查图像名称是否已经在 Excel 中
#                 if base_name not in set(worksheet['A']):
#                     worksheet.append([
#                         f"{base_name}_{i + 1}",
#                         box[0], box[1],  # 左上角顶点
#                         box[2], box[1],  # 右上角顶点
#                         box[2], box[3],  # 右下角顶点
#                         box[0], box[3]  # 左下角顶点
#                     ])

#             workbook.save(excel_file)


# def save_masked_image(image, mask, output_path, boxes):
#     plt.figure(figsize=(8, 8))
#     plt.imshow(image)
#     show_mask(mask, plt.gca())

#     # 绘制矩形框
#     # for box in boxes:
#     # plt.gca().add_patch(plt.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], fill=False, edgecolor='r', linewidth=2))
#     # plt.gca().add_patch(plt.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], fill=False,  linewidth=2))

#     plt.axis('off')
#     plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)
#     plt.close('all')
#     gc.collect()


# def show_mask(mask, ax, random_color=False):
#     color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0) if random_color else np.array(
#         [30 / 255, 144 / 255, 255 / 255, 1])
#     h, w = mask.shape[-2:]
#     mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
#     ax.imshow(mask_image)


# if __name__ == '__main__':
#     input_folder = "D://code//data//howtodo_Lv2//"
#     text = 'person,shoes'
#     process_images_in_folder_structure(input_folder, text)
#     print('完结完结完结')




In [ ]:
# 经过cursor内存优化后的代码
# 它获取的是三级文件夹中的图片

import os
import numpy as np
import torch
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision.ops import box_convert
from groundingdino.util.inference import load_model, load_image, predict, annotate
from segment_anything import sam_model_registry, SamPredictor
import openpyxl
from contextlib import contextmanager

# 忽略特定警告
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*device.*")
warnings.filterwarnings("ignore", category=UserWarning, message=".*requires_grad.*")

# 使用 GPU 设备（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 加载 GroundingDINO 模型
cfg_path = r'D://code//CV//GroundingDINO//groundingdino//config//GroundingDINO_SwinT_OGC.py'
weight_path = r'd://code//CV//GroundingDINO//weight//groundingdino_swint_ogc.pth'
gdino_model = load_model(cfg_path, weight_path)
gdino_model.to(device)
BOX_THRESHOLD = 0.35
TEXT_THRESHOLD = 0.35

# 加载 Segment Anything 模型
sam_checkpoint = r"D://aigc//ComfyUI_windows_portable//ComfyUI//models//sams//sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)

@contextmanager
def torch_gc():
    try:
        yield
    finally:
        gc.collect()
        torch.cuda.empty_cache()

def process_images_in_folder_structure(input_folder, text_prompt):
    total_files = 0
    processed_files = 0
    skipped_files = 0

    second_level_folders = [f.path for f in os.scandir(input_folder) if f.is_dir()]

    for second_folder in second_level_folders:
        print(f"Processing second level folder: {second_folder}")
        
        third_level_folders = [f.path for f in os.scandir(second_folder) if f.is_dir()]

        for third_folder in third_level_folders:
            print(f"Processing third level folder: {third_folder}")
            
            output_folder = os.path.join(third_folder, 'grounding_output')
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            image_files = [f for f in os.listdir(third_folder) if f.lower().endswith((".jpg", ".png"))]
            total_files += len(image_files)

            for filename in tqdm(image_files, desc=f"Processing {third_folder}"):
                ip = os.path.join(third_folder, filename)
                base_name = os.path.basename(ip).split('.')[0]
                output_filename = f"{base_name}.jpg"
                output_path = os.path.join(output_folder, output_filename)
                
                # 检查输出文件是否已存在
                if os.path.exists(output_path):
                    print(f"Skipping {filename} as it has already been processed.")
                    skipped_files += 1
                    continue

                try:
                    with torch_gc():
                        image, boxes = get_box_from_gdino_with_text_prompt(ip, text_prompt)
                        if image is not None and boxes is not None:
                            seg_with_sam_with_box_prompt(image, boxes, text_prompt, output_folder, ip)
                            processed_files += 1
                except Exception as e:
                    print(f"处理文件 {filename} 时出错: {e}")
                finally:
                    plt.close('all')
                    gc.collect()
                    torch.cuda.empty_cache()

    print(f"总文件数: {total_files}")
    print(f"成功处理的文件数: {processed_files}")
    print(f"跳过的文件数: {skipped_files}")


def get_box_from_gdino_with_text_prompt(ip, text_prompt):
    try:
        image_source, image = load_image(ip)
        if image_source is None or image is None:
            print(f"无法加载图像: {ip}")
            return None, None

        h, w = image_source.shape[:2]
        boxes, logits, phrases = predict(
            model=gdino_model,
            image=image,
            caption=text_prompt,
            box_threshold=BOX_THRESHOLD,
            text_threshold=TEXT_THRESHOLD,
            device=device
        )

        boxes = boxes.to(device)
        scale_tensor = torch.tensor([w, h, w, h], dtype=boxes.dtype, device=device)
        boxes = boxes * scale_tensor
        xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").cpu().numpy()

        del image, logits, phrases, boxes, scale_tensor
        gc.collect()
        torch.cuda.empty_cache()
        return image_source, xyxy
    except Exception as e:
        print(f"加载图像 {ip} 时出错: {e}")
        return None, None

def seg_with_sam_with_box_prompt(image, boxes, text_prompt, output_folder, ip):
    predictor = SamPredictor(sam)
    predictor.set_image(image)
    merged_mask = None

    for idx, box in enumerate(boxes):
        input_box = box.astype(int)
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=input_box[None, :],
            multimask_output=False,
        )
        if masks is not None and len(masks) > 0:
            if merged_mask is None:
                merged_mask = masks[0]
            else:
                merged_mask = np.logical_or(merged_mask, masks[0])

    if merged_mask is not None:
        merged_mask = np.where(merged_mask > 0, 1, 0)
        if np.any(merged_mask):
            base_name = os.path.basename(ip).split('.')[0]
            output_filename = f"{base_name}.jpg"
            output_path = os.path.join(output_folder, output_filename)
            save_masked_image(image, merged_mask, output_path, boxes)

            vertex_coords = [box for box in boxes]

            excel_file = os.path.join(output_folder, 'grounding_results.xlsx')
            if not os.path.exists(excel_file):
                workbook = openpyxl.Workbook()
                worksheet = workbook.active
                worksheet.append(['Image Name', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])
            else:
                workbook = openpyxl.load_workbook(excel_file)
                worksheet = workbook.active

            seen_vertices = set()
            for i, box in enumerate(boxes):
                if base_name not in set(worksheet['A']):
                    worksheet.append([
                        f"{base_name}_{i + 1}",
                        box[0], box[1],
                        box[2], box[1],
                        box[2], box[3],
                        box[0], box[3]
                    ])

            workbook.save(excel_file)

    del predictor, masks
    gc.collect()
    torch.cuda.empty_cache()

def save_masked_image(image, mask, output_path, boxes):
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    show_mask(mask, plt.gca())

    plt.axis('off')
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)
    plt.close('all')
    gc.collect()

def show_mask(mask, ax, random_color=False):
    color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0) if random_color else np.array(
        [30 / 255, 144 / 255, 255 / 255, 1])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

if __name__ == '__main__':
    input_folder = "D://code//data//howtodo_Lv2//"
    text = 'person,shoes'
    process_images_in_folder_structure(input_folder, text)
    print('完结完结完结')

    

Using device: cuda


c:\ProgramData\miniconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


c:\ProgramData\miniconda3\lib\site-packages\transformers\tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processing second level folder: D://code//data//howtodo_Lv2//女士春夏下装_from_0501
Processing third level folder: D://code//data//howtodo_Lv2//女士春夏下装_from_0501\9715


...

Processing D://code//data//howtodo_Lv2//男士春夏上装_from_0501\1349:  51%|█████     | 664/1314 [1:47:38<1:09:03,  6.37s/it]

处理文件 662cefcbF13f8ff00_377367e56c4ae1b2.jpg 时出错: Unable to allocate 19.5 MiB for an array with shape (800, 800, 4) and data type float64


Processing D://code//data//howtodo_Lv2//男士春夏上装_from_0501\1349:  51%|█████     | 665/1314 [1:47:44<1:09:07,  6.39s/it]

处理文件 662cf0c0F6abe9e34_cb06b36b94161a32.jpg 时出错: Unable to allocate 19.5 MiB for an array with shape (800, 800, 4) and data type float64


Processing D://code//data//howtodo_Lv2//男士春夏上装_from_0501\1349:  51%|█████     | 666/1314 [1:47:51<1:08:59,  6.39s/it]

# step3 - 将图片分为price/txt/white/scene四类
这里是Lv2期的特殊需求,针对单独的cid3进行分类,读取每个cid3中的grounding_output文件夹,在其中再进行四分类

In [ ]:
import os
import shutil
from PIL import Image
from paddleocr import PaddleOCR
from tqdm import tqdm
from datetime import datetime
import math
import time
import numpy as np
import pandas as pd

# 定义基础路径
base_path = 'D://code//data//howtodo_from_0401//测试1'

# 初始化 PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="ch", show_log=False)

def process_grounding_output(grounding_output_path):
    price_folder = os.path.join(grounding_output_path, 'price')
    txt_folder = os.path.join(grounding_output_path, 'txt')
    scene_folder = os.path.join(grounding_output_path, 'scene')
    white_folder = os.path.join(grounding_output_path, 'white')

    for folder in [price_folder, txt_folder, scene_folder, white_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)

    '''
    step1 - 筛选价促卖点图
    '''
    print(f'Step 1: 筛选价促卖点图 - {grounding_output_path}')

    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]

    with tqdm(total=len(image_files), desc="Processing images") as pbar:
        for filename in image_files:
            img_path = os.path.join(grounding_output_path, filename)
            
            result = ocr.ocr(img_path, cls=True)
            
            if not result or not result[0]:
                pbar.update(1)
                continue
            
            data_list = result[0]
            
            keywords = ['满', '减', '折', '到手价', '送', '免息', '活动价', '包邮价', '参考价', 
                        r'.*满.*减.*', r'.*满.*-.*', r'.*满.*赠.*',r'.*满.*送.*', r'.*价.*', 
                        '券', '优惠', '用券', '领券', '券', '送', '低至', '立减', '直降', '免息',
                        '¥', '夫', '￥']
            
            if any(keyword in data[1][0] for data in data_list for keyword in keywords):
                try:
                    shutil.move(img_path, price_folder)
                    pbar.write(f"Image '{filename}' moved to price_folder.")
                except Exception as e:
                    pbar.write(f"Failed to move image '{filename}': {e}")
            
            pbar.update(1)

    '''
    step2 - 筛选白底图
    '''
    print(f'Step 2: 筛选白底图 - {grounding_output_path}')

    def move_images_with_white_pixels(src_folder, dst_folder, threshold=0.40):
        image_files = [f for f in os.listdir(src_folder) if f.endswith(('.jpg', '.png'))]
        
        with tqdm(total=len(image_files), desc=f"Moving images from {src_folder}") as pbar:
            for filename in image_files:
                img_path = os.path.join(src_folder, filename)
                
                with Image.open(img_path) as img:
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                    width, height = img.size
                
                    white_pixels = sum(1 for x in range(width) for y in range(height) 
                                       if all(v > 230 for v in img.getpixel((x, y))[:3]))
                    
                    if white_pixels / (width * height) > threshold:
                        try:
                            shutil.move(img_path, dst_folder)
                            pbar.write(f"Image '{filename}' moved to white folder.")
                        except Exception as e:
                            pbar.write(f"Error moving image '{filename}': {e}")
                            
                pbar.update(1)

    move_images_with_white_pixels(grounding_output_path, white_folder, threshold=0.40)

    '''
    step3 - 筛选功能卖点图
    '''
    print(f'Step 3: 筛选功能卖点图 - {grounding_output_path}')

    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]
    for filename in image_files:
        img_path = os.path.join(grounding_output_path, filename)
        
        img = Image.open(img_path)
        width, height = img.size
        
        cropped_img = img.crop((0, height // 6, width, height))
        img_np = np.array(cropped_img)
        
        result = ocr.ocr(img_np, cls=True)
        
        if result and result[0] and len(result[0]) >= 2:
            shutil.move(img_path, txt_folder)
            print(f"'{filename}' moved to txt folder.")
        else:
            print(f"'{filename}' remains in source folder.")

    '''
    step4 - 将剩余图片归类到scene
    '''
    print(f'Step 4: 归类图片到scene - {grounding_output_path}')

    image_files = [f for f in os.listdir(grounding_output_path) if f.endswith(('.jpg', '.png'))]
    for filename in image_files:
        img_path = os.path.join(grounding_output_path, filename)
        shutil.move(img_path, scene_folder)
        print(f"Image '{filename}' moved to scene_folder.")

    '''
    step5 - 将white中有文本的图片移到txt_folder
    '''
    print(f'Step 5: 将white中有文本的图片移到txt_folder - {grounding_output_path}')

    image_files = [f for f in os.listdir(white_folder) if f.endswith(('.jpg', '.png'))]

    with tqdm(total=len(image_files), desc="Processing images") as pbar:
        for filename in image_files:
            img_path = os.path.join(white_folder, filename)

            img = Image.open(img_path)
            width, height = img.size
        
            cropped_img = img.crop((0, height // 6, width, height))
            img_np = np.array(cropped_img)
            
            result = ocr.ocr(img_np, cls=True)
            
            if result and result[0]:
                lines_above_30_count = sum(1 for rectangle in result[0] 
                                           if min(math.sqrt((x2-x1)**2 + (y2-y1)**2) 
                                                  for (x1, y1), (x2, y2) in zip(rectangle[0], rectangle[0][1:])) > 30)
                
                if lines_above_30_count >= 2:
                    shutil.move(img_path, txt_folder)
                    pbar.write(f"Image '{filename}' moved to txt_folder.")
                else:
                    pbar.write(f"Image '{filename}' remains in source folder.")
            
            pbar.update(1)

def main():
    for root, dirs, files in os.walk(base_path):
        if 'grounding_output' in dirs:
            grounding_output_path = os.path.join(root, 'grounding_output')
            print(f"Processing: {grounding_output_path}")
            process_grounding_output(grounding_output_path)

    print("All processing completed.")
    print("当前系统时间是:", datetime.now())

if __name__ == '__main__':
    main()

    